In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split, RandomizedSearchCV
import joblib
from xgboost import XGBRegressor
from scipy.stats import randint, uniform

# Load the data
input_csv_path = 'compounds_with_predictions2.csv'  # Replace with your actual CSV file path
df = pd.read_csv(input_csv_path)

# Convert MACCS fingerprints from comma-separated string to a list of integers
def maccs_to_array(maccs_str):
    try:
        # Convert the string to a list of integers and discard the first bit (bit 0)
        return np.array(list(map(int, maccs_str.split(',')))[1:], dtype=int)
    except ValueError:
        return np.zeros(166, dtype=int)  # Default to an array of zeros if there's an error

# Apply the function to convert the MACCS fingerprint strings
df['MACCS_fingerprint'] = df['MACCS_fingerprint'].apply(maccs_to_array)

# Convert MACCS fingerprints to a feature matrix
X = np.array(df['MACCS_fingerprint'].tolist())

# Extract pIC50 values (target variable y)
y = df['pIC50'].values

# Use all features after discarding the first bit
X_filtered = X

# Initialize MinMaxScaler for scaling target variable
scaler_y = MinMaxScaler()
y_transformed = scaler_y.fit_transform(y.reshape(-1, 1)).flatten()

# Split the data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X_filtered, y_transformed, test_size=0.2, random_state=42)

# Initialize StandardScaler for feature scaling
scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

# Initialize the XGBRegressor
xgb = XGBRegressor(random_state=42, use_label_encoder=False, eval_metric='rmse')

# Define the parameter grid for RandomizedSearchCV
param_distributions = {
    'n_estimators': randint(100, 1000),  # Start from a higher number to avoid very low values
    'learning_rate': uniform(0.01, 0.3),
    'max_depth': randint(4, 10),  # Slightly narrower range
    'subsample': uniform(0.7, 0.3),
    'colsample_bytree': uniform(0.7, 0.3),
    'reg_alpha': uniform(0.0, 1.0),  # L1 regularization
    'reg_lambda': uniform(0.0, 1.0)  # L2 regularization
}

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_distributions,
    n_iter=200,  # Number of parameter settings sampled
    cv=15,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    random_state=42,
    verbose=2
)

# Perform random search to find the best hyperparameters
random_search.fit(X_train_scaled, y_train)

# Retrieve the best model
best_model = random_search.best_estimator_

# Predict transformed pIC50 values for the test set
y_pred_transformed = best_model.predict(X_test_scaled)
y_pred = scaler_y.inverse_transform(y_pred_transformed.reshape(-1, 1)).flatten()
y_test_original = scaler_y.inverse_transform(y_test.reshape(-1, 1)).flatten()

# Evaluate the model on the test data
mse_test = mean_squared_error(y_test_original, y_pred)
rmse_test = np.sqrt(mse_test)
mae_test = mean_absolute_error(y_test_original, y_pred)
r2_test = r2_score(y_test_original, y_pred)

# Predict transformed pIC50 values for the training set
y_train_pred_transformed = best_model.predict(X_train_scaled)
y_train_pred = scaler_y.inverse_transform(y_train_pred_transformed.reshape(-1, 1)).flatten()

# Evaluate the model on the training data
mse_train = mean_squared_error(y_train, y_train_pred_transformed)
rmse_train = np.sqrt(mse_train)
mae_train = mean_absolute_error(y_train, y_train_pred_transformed)
r2_train = r2_score(y_train, y_train_pred_transformed)

# Print evaluation metrics for test and training sets
print(f"Test set - Mean Squared Error (MSE): {mse_test:.4f}")
print(f"Test set - Root Mean Squared Error (RMSE): {rmse_test:.4f}")
print(f"Test set - Mean Absolute Error (MAE): {mae_test:.4f}")
print(f"Test set - R-squared (R2): {r2_test:.4f}")

print(f"Training set - Mean Squared Error (MSE): {mse_train:.4f}")
print(f"Training set - Root Mean Squared Error (RMSE): {rmse_train:.4f}")
print(f"Training set - Mean Absolute Error (MAE): {mae_train:.4f}")
print(f"Training set - R-squared (R2): {r2_train:.4f}")


# Save the best model and scalers for future use
joblib.dump(best_model, 'xgb_model_filtered.pkl')
joblib.dump(scaler_y, 'y_scaler_filtered.pkl')
joblib.dump(scaler_X, 'X_scaler_filtered.pkl')


Fitting 15 folds for each of 200 candidates, totalling 3000 fits


c:\Users\MK\anaconda3\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


Test set - Mean Squared Error (MSE): 0.4994
Test set - Root Mean Squared Error (RMSE): 0.7067
Test set - Mean Absolute Error (MAE): 0.5194
Test set - R-squared (R2): 0.6005
Training set - Mean Squared Error (MSE): 0.0024
Training set - Root Mean Squared Error (RMSE): 0.0492
Training set - Mean Absolute Error (MAE): 0.0324
Training set - R-squared (R2): 0.9429


['X_scaler_filtered.pkl']

In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from xgboost import XGBRegressor
from scipy.stats import randint, uniform
import joblib

# Load the data
input_csv_path = 'compounds_with_predictions2.csv'  # Replace with your actual CSV file path
df = pd.read_csv(input_csv_path)

# Convert MACCS fingerprints from comma-separated string to a list of integers
def maccs_to_array(maccs_str):
    try:
        # Convert the string to a list of integers and discard the first bit (bit 0)
        return np.array(list(map(int, maccs_str.split(',')))[1:], dtype=int)
    except ValueError:
        return np.zeros(166, dtype=int)  # Default to an array of zeros if there's an error

# Apply the function to convert the MACCS fingerprint strings
df['MACCS_fingerprint'] = df['MACCS_fingerprint'].apply(maccs_to_array)

# Convert MACCS fingerprints to a feature matrix
X = np.array(df['MACCS_fingerprint'].tolist())

# Extract pIC50 values (target variable y)
y = df['pIC50'].values

# Use all features after discarding the first bit
X_filtered = X

# Initialize MinMaxScaler for scaling target variable
scaler_y = MinMaxScaler()
y_transformed = scaler_y.fit_transform(y.reshape(-1, 1)).flatten()

# Split the data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X_filtered, y_transformed, test_size=0.2, random_state=42)

# Initialize StandardScaler for feature scaling
scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

# Initialize the XGBRegressor with early stopping and learning curve improvements
xgb = XGBRegressor(random_state=42, objective='reg:squarederror', eval_metric='rmse')

# Define the parameter grid for RandomizedSearchCV
param_distributions = {
    'n_estimators': randint(100, 1500),  # Increased upper limit for more complex models
    'learning_rate': uniform(0.01, 0.3),
    'max_depth': randint(3, 12),  # Broadened range for depth exploration
    'subsample': uniform(0.5, 0.5),  # Allowing more variation in subsampling
    'colsample_bytree': uniform(0.5, 0.5),  # Variability in column subsampling
    'gamma': uniform(0, 5),  # Add gamma regularization parameter to prevent overfitting
    'min_child_weight': randint(1, 10),  # Tuning for preventing overfitting
    'reg_alpha': uniform(0.0, 1.0),  # L1 regularization
    'reg_lambda': uniform(0.0, 1.0)  # L2 regularization
}

# Initialize RandomizedSearchCV
random_search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_distributions,
    n_iter=300,  # Increase the number of iterations for broader hyperparameter search
    cv=10,  # 10-fold cross-validation for robust estimation
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    random_state=42,
    verbose=2
)

# Perform random search to find the best hyperparameters
random_search.fit(X_train_scaled, y_train, eval_set=[(X_test_scaled, y_test)], early_stopping_rounds=10, verbose=False)

# Retrieve the best model
best_model = random_search.best_estimator_

# Predict transformed pIC50 values for the test set
y_pred_transformed = best_model.predict(X_test_scaled)
y_pred = scaler_y.inverse_transform(y_pred_transformed.reshape(-1, 1)).flatten()
y_test_original = scaler_y.inverse_transform(y_test.reshape(-1, 1)).flatten()

# Evaluate the model on the test data
mse_test = mean_squared_error(y_test_original, y_pred)
rmse_test = np.sqrt(mse_test)
mae_test = mean_absolute_error(y_test_original, y_pred)
r2_test = r2_score(y_test_original, y_pred)

# Predict transformed pIC50 values for the training set
y_train_pred_transformed = best_model.predict(X_train_scaled)
y_train_pred = scaler_y.inverse_transform(y_train_pred_transformed.reshape(-1, 1)).flatten()

# Evaluate the model on the training data
mse_train = mean_squared_error(y_train, y_train_pred_transformed)
rmse_train = np.sqrt(mse_train)
mae_train = mean_absolute_error(y_train, y_train_pred_transformed)
r2_train = r2_score(y_train, y_train_pred_transformed)

# Print evaluation metrics for test and training sets
print(f"Test set - Mean Squared Error (MSE): {mse_test:.4f}")
print(f"Test set - Root Mean Squared Error (RMSE): {rmse_test:.4f}")
print(f"Test set - Mean Absolute Error (MAE): {mae_test:.4f}")
print(f"Test set - R-squared (R2): {r2_test:.4f}")

print(f"Training set - Mean Squared Error (MSE): {mse_train:.4f}")
print(f"Training set - Root Mean Squared Error (RMSE): {rmse_train:.4f}")
print(f"Training set - Mean Absolute Error (MAE): {mae_train:.4f}")
print(f"Training set - R-squared (R2): {r2_train:.4f}")

# Save the best model and scalers for future use
joblib.dump(best_model, 'xgb_model_filtered.pkl')
joblib.dump(scaler_y, 'y_scaler_filtered.pkl')
joblib.dump(scaler_X, 'X_scaler_filtered.pkl')


Fitting 10 folds for each of 300 candidates, totalling 3000 fits
Test set - Mean Squared Error (MSE): 0.4887
Test set - Root Mean Squared Error (RMSE): 0.6990
Test set - Mean Absolute Error (MAE): 0.5180
Test set - R-squared (R2): 0.6091
Training set - Mean Squared Error (MSE): 0.0104
Training set - Root Mean Squared Error (RMSE): 0.1018
Training set - Mean Absolute Error (MAE): 0.0773
Training set - R-squared (R2): 0.7556


c:\Users\MK\anaconda3\Lib\site-packages\xgboost\sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


['X_scaler_filtered.pkl']

In [4]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from xgboost import XGBRegressor
from scipy.stats import randint, uniform
import joblib

# Load the data
input_csv_path = 'compounds_with_predictions2.csv'  # Replace with your actual CSV file path
df = pd.read_csv(input_csv_path)

# Convert MACCS fingerprints from comma-separated string to a list of integers
def maccs_to_array(maccs_str):
    try:
        # Convert the string to a list of integers and discard the first bit (bit 0)
        return np.array(list(map(int, maccs_str.split(',')))[1:], dtype=int)
    except ValueError:
        return np.zeros(166, dtype=int)  # Default to an array of zeros if there's an error

# Apply the function to convert the MACCS fingerprint strings
df['MACCS_fingerprint'] = df['MACCS_fingerprint'].apply(maccs_to_array)

# Convert MACCS fingerprints to a feature matrix
X = np.array(df['MACCS_fingerprint'].tolist())

# Extract pIC50 values (target variable y)
y = df['pIC50'].values

# Initialize MinMaxScaler for scaling target variable
scaler_y = MinMaxScaler()
y_transformed = scaler_y.fit_transform(y.reshape(-1, 1)).flatten()

# Split the data into training and testing sets (80% training, 20% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y_transformed, test_size=0.2, random_state=42)

# Initialize StandardScaler for feature scaling
scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train)
X_test_scaled = scaler_X.transform(X_test)

# Initialize the XGBRegressor with early stopping
xgb = XGBRegressor(random_state=42, objective='reg:squarederror', eval_metric='rmse')

# Define the parameter grid for RandomizedSearchCV with adjusted regularization and complexity
param_distributions = {
    'n_estimators': randint(50, 300),  # Reduce the number of estimators to prevent overfitting
    'learning_rate': uniform(0.01, 0.1),  # Lower learning rates for more stable learning
    'max_depth': randint(3, 6),  # Limit model complexity by reducing depth
    'subsample': uniform(0.5, 0.9),  # Slightly reduce subsampling
    'colsample_bytree': uniform(0.5, 0.9),  # Reduce feature subsampling to avoid overfitting
    'gamma': uniform(1, 5),  # Increase gamma to enforce further regularization
    'min_child_weight': randint(5, 15),  # Higher values force the model to be less sensitive to small data variations
    'reg_alpha': uniform(0.5, 2.0),  # L1 regularization to reduce complexity
    'reg_lambda': uniform(1.0, 3.0)  # Stronger L2 regularization
}

# Initialize RandomizedSearchCV with more conservative hyperparameter ranges
random_search = RandomizedSearchCV(
    estimator=xgb,
    param_distributions=param_distributions,
    n_iter=200,  # Fewer iterations for faster tuning
    cv=10,  # 10-fold cross-validation
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    random_state=42,
    verbose=2
)

# Perform random search with early stopping
random_search.fit(X_train_scaled, y_train, eval_set=[(X_test_scaled, y_test)], early_stopping_rounds=10, verbose=True)

# Retrieve the best model
best_model = random_search.best_estimator_

# Predict transformed pIC50 values for the test set
y_pred_transformed = best_model.predict(X_test_scaled)
y_pred = scaler_y.inverse_transform(y_pred_transformed.reshape(-1, 1)).flatten()
y_test_original = scaler_y.inverse_transform(y_test.reshape(-1, 1)).flatten()

# Evaluate the model on the test data
mse_test = mean_squared_error(y_test_original, y_pred)
rmse_test = np.sqrt(mse_test)
mae_test = mean_absolute_error(y_test_original, y_pred)
r2_test = r2_score(y_test_original, y_pred)

# Predict transformed pIC50 values for the training set
y_train_pred_transformed = best_model.predict(X_train_scaled)
y_train_pred = scaler_y.inverse_transform(y_train_pred_transformed.reshape(-1, 1)).flatten()

# Evaluate the model on the training data
mse_train = mean_squared_error(y_train, y_train_pred_transformed)
rmse_train = np.sqrt(mse_train)
mae_train = mean_absolute_error(y_train, y_train_pred_transformed)
r2_train = r2_score(y_train, y_train_pred_transformed)

# Print evaluation metrics for test and training sets
print(f"Test set - Mean Squared Error (MSE): {mse_test:.4f}")
print(f"Test set - Root Mean Squared Error (RMSE): {rmse_test:.4f}")
print(f"Test set - Mean Absolute Error (MAE): {mae_test:.4f}")
print(f"Test set - R-squared (R2): {r2_test:.4f}")

print(f"Training set - Mean Squared Error (MSE): {mse_train:.4f}")
print(f"Training set - Root Mean Squared Error (RMSE): {rmse_train:.4f}")
print(f"Training set - Mean Absolute Error (MAE): {mae_train:.4f}")
print(f"Training set - R-squared (R2): {r2_train:.4f}")

# Save the best model and scalers for future use
joblib.dump(best_model, 'xgb_model_filtered.pkl')
joblib.dump(scaler_y, 'y_scaler_filtered.pkl')
joblib.dump(scaler_X, 'X_scaler_filtered.pkl')


Fitting 10 folds for each of 200 candidates, totalling 2000 fits
[0]	validation_0-rmse:0.22172
[1]	validation_0-rmse:0.21964
[2]	validation_0-rmse:0.21623
[3]	validation_0-rmse:0.21471
[4]	validation_0-rmse:0.21470
[5]	validation_0-rmse:0.21193
[6]	validation_0-rmse:0.20945
[7]	validation_0-rmse:0.20945
[8]	validation_0-rmse:0.20945
[9]	validation_0-rmse:0.20777
[10]	validation_0-rmse:0.20638
[11]	validation_0-rmse:0.20638
[12]	validation_0-rmse:0.20638
[13]	validation_0-rmse:0.20638
[14]	validation_0-rmse:0.20638
[15]	validation_0-rmse:0.20638
[16]	validation_0-rmse:0.20638
[17]	validation_0-rmse:0.20638
[18]	validation_0-rmse:0.20638


c:\Users\MK\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
1460 fits failed out of a total of 2000.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\MK\anaconda3\Lib\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\MK\anaconda3\Lib\site-packages\xgboost\core.py", line 730, in inner_f
    return func(**kwargs)
           ^^^^^^^^^^^^^^
  File "c:\Users\MK\anaconda3\Lib\site-packages\xgboost\sklearn.py", line 1090, in fit
    self._Booster = train(
                    ^^^^^^
  File "c:\Users\MK\anaconda3\Li

[19]	validation_0-rmse:0.20639
Test set - Mean Squared Error (MSE): 1.0542
Test set - Root Mean Squared Error (RMSE): 1.0267
Test set - Mean Absolute Error (MAE): 0.8580
Test set - R-squared (R2): 0.1567
Training set - Mean Squared Error (MSE): 0.0364
Training set - Root Mean Squared Error (RMSE): 0.1907
Training set - Mean Absolute Error (MAE): 0.1617
Training set - R-squared (R2): 0.1412


['X_scaler_filtered.pkl']

In [3]:
import numpy as np
import pandas as pd
from rdkit import Chem
from rdkit.Chem import MACCSkeys
import joblib

# Load the saved XGBoost model and scalers
best_model = joblib.load('xgb_model_filtered.pkl')  # Load XGBoost model
scaler_y = joblib.load('y_scaler_filtered.pkl')  # Load y-scaler
scaler_X = joblib.load('X_scaler_filtered.pkl')  # Load X-scaler

# Define a function to convert SMILES to MACCS fingerprints and discard the first bit
def smiles_to_maccs(smiles):
    molecule = Chem.MolFromSmiles(smiles)
    if molecule is None:
        return np.zeros(166, dtype=int)  # Return a default array of 166 bits if SMILES is invalid
    maccs_fingerprint = MACCSkeys.GenMACCSKeys(molecule)
    return np.array(list(maccs_fingerprint)[1:], dtype=int)  # Convert to 166-bit fingerprint (discarding the first bit)

# Example list of new SMILES strings
smiles_list = [
    'C=C[C@@](C)(O)C[C@@H](C)[C@H]4CCC3C2CCC1C(=C)[C@@H](O)CC[C@]1(C)C2CC[C@]3(C)C4'
]

# Convert SMILES strings to MACCS fingerprints
fingerprints = np.array([smiles_to_maccs(smiles) for smiles in smiles_list])

# Ensure the selection filter aligns with the training step
# std_devs = np.std(fingerprints, axis=0)
# sd_cutoff = 0.1
# selected_features = std_devs > sd_cutoff  # Use the same cutoff used during training

# Filter and scale the new data
# fingerprints_filtered = fingerprints[:, selected_features]  # Apply the same filter used during training
fingerprints_filtered = fingerprints  # No filtering applied
X_new_scaled = scaler_X.transform(fingerprints_filtered)

# Predict using the loaded model
y_pred_transformed = best_model.predict(X_new_scaled)
y_pred = scaler_y.inverse_transform(y_pred_transformed.reshape(-1, 1)).flatten()

# Print predictions
for smiles, prediction in zip(smiles_list, y_pred):
    print(f"SMILES: {smiles}, Predicted pIC50: {prediction:.4f}")
    predicted_IC50 = 10 ** (-prediction) * 1000000
    print(f"Predicted IC50: {predicted_IC50:.4f} uM")


SMILES: C=C[C@@](C)(O)C[C@@H](C)[C@H]4CCC3C2CCC1C(=C)[C@@H](O)CC[C@]1(C)C2CC[C@]3(C)C4, Predicted pIC50: 5.2762
Predicted IC50: 5.2939 uM
